# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [60]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report




In [73]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('Msg_Category', engine)
all_cols = df.columns.to_list()
x_cols = all_cols[0:3]
y_cols =  all_cols
del y_cols[0:3]

df_X = df[x_cols]
df_Y = df[y_cols]

X = df_X.message.values
y = df_Y.values

In [74]:
y.shape

(26180, 35)

### 2. Write a tokenization function to process your text data

In [75]:
def tokenize(text):
    '''
    tokenize
    normalise words of a sentence by preprocessing it intro individual words.
     Input:
     text - A sentence string with disaster message 
     Output:
     words: A list of pre-processed words
    '''
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    words = [WordNetLemmatizer().lemmatize(w) for w in words]
    words = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=43)

In [80]:
pipeline = sklearn.pipeline.Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                                        ('tfidf', TfidfTransformer()),
                                        ('clf', MultiOutputClassifier(LogisticRegression()))])
                                        
parameters = {'clf__estimator__C': [0.5,1.0,1.5,2,2.5,3],
'clf__estimator__max_iter': [100, 400]}


grid_search = GridSearchCV(pipeline, param_grid=parameters, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_



In [85]:
best_params

{'clf__estimator__C': 3, 'clf__estimator__max_iter': 400}

In [76]:
#best_params = {'clf__estimator__C': 1.5, 'clf__estimator__max_iter': 400}
#pipeline = sklearn.pipeline.Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
#('tfidf', TfidfTransformer()),
#('clf', MultiOutputClassifier(LogisticRegression()))])
#pipeline.set_params(**best_params)

In [78]:
for i in range(0,35):
    if (sum(y_test[:,i]==1) == 0) | (sum(y_test[:,i]==1) == 0):
        print(i)

In [81]:
labels = np.unique(y_pred)
accuracy = (y_pred == y_test).mean()

print("Labels:", labels)
print("Accuracy:", accuracy)

Labels: [0 1]
Accuracy: 0.948234126984127


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [87]:
y_pred = grid_search.predict(X_test)

Precision = []
Recall = []
F1 = []
Acc = []

for i in range(y_test.shape[1]):
    report = classification_report(y_test[:, i], y_pred[:, i], output_dict=True)
    
    macro_precision = report['macro avg']['precision']
    Precision.append(macro_precision)
    
    macro_recall = report['macro avg']['recall']
    Recall.append(macro_recall)
    
    macro_f1_score = report['macro avg']['f1-score']
    F1.append(macro_f1_score)

    accuracy = report['accuracy']
    Acc.append(accuracy)


cats = df_Y.columns.to_list()

Evaluate_1 = pd.DataFrame(
    {'Category': cats,
     'Precision' : Precision,
     'Recall' : Recall,
     'Accuracy': Acc,
     'F1': F1
    })

Evaluate_1

c:\Users\avadi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avadi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avadi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avadi\anaconda3\lib\site-packages\sklearn\metr

,Category,Precision,Recall,Accuracy,F1
0,related,0.780047,0.712686,0.832060,0.735811
1,request,0.837436,0.767002,0.896875,0.795381
2,offer,0.498206,0.500000,0.996412,0.499101
3,aid_related,0.762226,0.755720,0.768519,0.758258
4,medical_help,0.790863,0.628212,0.931019,0.671324
5,medical_products,0.812862,0.623832,0.960417,0.673118
6,search_and_rescue,0.837965,0.558861,0.974653,0.595379
7,security,0.740449,0.502953,0.980787,0.501102
8,military,0.822611,0.602341,0.971644,0.651535
9,water,0.864016,0.757682,0.959606,0.800274


### 6. Improve your model
Use grid search to find better parameters. 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

python models/train_classifier.py data/DisasterResponse.db models/model.pkl